# Data Collection through Teleoperation

The code is modified from NVIDIA-AI-IOT/jetbot-GitHub.\
In this notebook we'll control the Jetbot remotely with a gamepad controller connected to our web browser machine to take a snapshot and moving around.\
\
本程式碼改寫自NVIDIA-AI-IOT/jetbot-GitHub\
使用本程式碼可透過控制器(gamepad)遠端控制jetbot蒐集照片資料

### Create gamepad controller

將控制器USB端接到``電腦``(注意：非接到jetson nano)，開啟以下網頁連結測試控制器並紀錄控制器參數\
接下來將會建立一個控制jetbot的``Controller``物件，其中``index``為控制器編號，若有多台控制器需注意編號以免建立物件時產生錯誤

1. 進入以下連結 [http://html5gamepad.com](http://html5gamepad.com).  
2. 按下控制器上的按鍵了解控制器對應參數
3. 記下控制器編號以便建立控制器物件


In [2]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # 填入您的控制器的index

display(controller)  # 實時顯示控制器面板

Controller()

第一次使用時，若填入的index正確且顯示``Connect gamepad and press any button``，則隨意按下任何控制器按鍵，控制器面板就會顯示在螢幕上

### Connect gamepad controller to robot motors

建立控制器連線後，接下來要建立控制器與馬達的連線\
這邊將控制器以中心分為左右，左邊搖桿/方向鍵(controller.axes[1])控制前後，右邊搖桿/控制鍵(controller.buttons[3]/[1])分別控制左右馬達\
修改lamda函數x的倍數可以更改馬達輸出力度(例：0.2代表馬達輸出20%馬力)

In [19]:
from jetbot import Robot
import traitlets

robot = Robot()

forward_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -0.2*x)
backward_link = traitlets.dlink((controller.axes[1], 'value'), (robot.right_motor, 'value'), transform=lambda x: -0.2*x)
left_link = traitlets.dlink((controller.buttons[3], 'value'), (robot.left_motor, 'value'), transform=lambda x: -0.2*x)
right_link = traitlets.dlink((controller.buttons[1], 'value'), (robot.right_motor, 'value'), transform=lambda x: -0.2*x)

### Create and display Image widget

接下來我們將建立相機的物件，並將相機取得的畫面實時顯示在螢幕上

1. 首先建立camera的物件並調整長寬屬性
2. 建立Image widget物件並將長寬調整和相機物件相同，檔案格式設定成jpeg
3. 建立camera與image widget的連結(因camera取得的影像數值為BGR8(blue, green, red, 8bit)而image接受的影像為JPEG, 故需做transform轉換)

In [3]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'', format='jpeg', height='224', width='224')

進行到這步時，我們就可以坐在電腦前看著jetbot相機的實時畫面並用控制器遠端控制jetbot行動

### Stop robot if network disconnects

運行此程式碼區塊後，若不慎發生jetbot wifi斷線時，將自動切斷jetbot與控制器連線並停止馬達轉動，避免jetbot撞牆或墜落

In [6]:
from jetbot import Heartbeat


def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

### Reconnecting to the robot
當發生wifi斷線jetbot連線中止後，可執行以下區塊程式碼恢復連線\
但若未發生前述情況不要請不要執行以下程式碼，避免造成多重連線問題；多重連線會導致後續一次拍攝多張重複照片

In [7]:
# 先不要執行, 若連結斷掉再執行, 不然會出現重複多連結問題

forward_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -0.2x)
backward_link = traitlets.dlink((controller.axes[1], 'value'), (robot.right_motor, 'value'), transform=lambda x: -0.2x)
left_link = traitlets.dlink((controller.axes[0], 'value'), (robot.left_motor, 'value'), transform=lambda x: -0.2x)
right_link = traitlets.dlink((controller.axes[0], 'value'), (robot.right_motor, 'value'), transform=lambda x: -0.2x)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### Save snapshots with gamepad button

確認相機串流畫面沒有問題後，接下來我們將使用控制器按鈕來控制jetbot執行拍照動作\
\
本專案會需要蒐集三種類型照片：
1. 障礙物照片
2. 可行走空間照片
3. 蟑螂照片
blocked以及free照片用作jebot避障訓練，cockroach用作物體追蹤訓練

因此，我們先建立三個資料夾，分別命名為blocked、free、cockroach，且將三個資料夾都置於datasets資料夾下方便管理


In [4]:
import os

cockroach_dir = './dataset/cockroach'
blocked_dir = './dataset/blocked'
free_dir = './dataset/free'

try:
    os.makedirs(cockroach_dir)
    os.makedirs(blocked_dir)
    os.makedirs(free_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


接下來，我們分別定義三個函數save_cockroach, save_blocked, save_free來擷取照片\
並使用uuid套件給予照片流水號，使之名稱不重複

In [ ]:
import uuid

### cockroach
cockroach_image = widgets.Image(format='jpeg', width=224, height=224)

def save_cockroach(change):
    # 當按鍵按下時會存照片
    if change['new']:
        file_path = './dataset/cockroach/' + str(uuid.uuid1()) + '.jpg'
        
        with open(file_path, 'wb') as f:  # 存jpeg格式的照片
            f.write(image.value)
            
        cockroach_image.value = image.value  # 實時顯示存下的照片
        cockroach_count.value = len(os.listdir(cockroach_dir))  # 計算已拍照片張數

        
### blocked
blocked_image = widgets.Image(format='jpeg', width=224, height=224)

def save_blocked(change):
    if change['new']:
        file_path = './dataset/blocked/' + str(uuid.uuid1()) + '.jpg'

        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        blocked_image.value = image.value
        blocked_count.value = len(os.listdir(blocked_dir))
        

### free
free_image = widgets.Image(format='jpeg', width=224, height=224)

def save_free(change):
    if change['new']:
        file_path = './dataset/free/' + str(uuid.uuid1()) + '.jpg'
        
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        free_image.value = image.value
        free_count.value = len(os.listdir(free_dir))

        
# 建立已拍照片張數實時顯示窗格
button_layout = widgets.Layout(width='256px', height='64px')

cockroach_count = widgets.IntText(layout=button_layout, value=len(os.listdir(cockroach_dir)), description='cockroach')
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)), description='blocked')
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)), description='free')


# 連動控制器按鈕及自定義的拍照函數
controller.buttons[5].observe(save_cockroach, names='value')
controller.buttons[7].observe(save_blocked, names='value')
controller.buttons[6].observe(save_free, names='value')
        
# 實時顯示相機串流畫面、各類最近一張拍攝照片及總拍攝張數
display(widgets.HBox([free_count, blocked_count, cockroach_count]))
display(widgets.HBox([image, free_image, blocked_image, cockroach_image]))


照片資料收集完成後記得要關閉相機連線，以方便下次使用

In [ ]:
camera.stop()

## Next

照片收集完成後，使用linux作業系統的zip指令壓縮打包dataset資料夾\
接著將dataset.zip複製到另外有GPU的主機或是雲端上進行訓練(在jetson nano 2GB上訓練可能會顯示記憶體不足)

> ! 表示使用shell command\
> -r 表示recursive，加入此選項會將所有巢狀結構下資料夾都壓縮打包\
> -q 表示quiet，加入此選項在執行壓縮打包時將不會印出訊息


In [ ]:
!zip -r -q dataset.zip dataset

執行到此步驟在資料夾中應該可以看到dataset.zip檔了，可以將它下載到本地主機中或上傳雲端進行後續訓練

### Conclusion

本程式碼到此告一段落，若有疑問還請email聯繫，謝謝